**Download data**

restaurant_review.txt - the text corpus for pretrain Albert, used for building vocab too

albert_config.json - Albert model configuration, specifies e.g. the model hidden layer size, # of vocab etc





In [0]:
!pip install tensorflow==1.14.0
!wget https://github.com/LydiaXiaohongLi/Albert_Finetune_with_Pretrain_on_Custom_Corpus/raw/master/data_toy/restaurant_review.txt
!wget https://github.com/LydiaXiaohongLi/Albert_Finetune_with_Pretrain_on_Custom_Corpus/raw/master/models_toy/albert_config.json

     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 491kB 44.7MB/s 
     |████████████████████████████████| 3.2MB 44.8MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
--2020-06-03 16:03:48--  https://github.com/LydiaXiaohongLi/Albert_Finetune_with_Pretrain_on_Custom_Corpus/raw/master/data_toy/restaurant_review.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LydiaXiaohongLi/Albert_Finetune_with_Pretra

**Build Vocab**

We use the open sorce implementation at https://github.com/kwonmha/bert-vocab-builder.git for building vocab from our pretrain corpus. Which reads the text corpus and identifies frequent word and subword tokens, and add special chars and other special tokens required by bert/albert model


In [0]:
!git clone https://github.com/kwonmha/bert-vocab-builder.git
!python ./bert-vocab-builder/subword_builder.py --corpus_filepattern "restaurant_review.txt" --output_filename "vocab.txt" --min_count 1

Cloning into 'bert-vocab-builder'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 91 (delta 0), reused 1 (delta 0), pack-reused 88
Unpacking objects: 100% (91/91), done.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of typ

**Create Pretrain files**

Create the Albert pretraining input files, tokenize the text corpus sentences, encode the tokens (into vocab file index for that particular token), create random masked input sentences, create random sentences pair from text corpus etc, for pretrain Albert model

In [0]:
!pip install sentencepiece
!pip install albert-tensorflow
!git clone https://github.com/google-research/albert.git

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 81kB 2.2MB/s 
Cloning into 'albert'...
remote: Enumerating objects: 347, done.
remote: Total 347 (delta 0), reused 0 (delta 0), pack-reused 347
Receiving objects: 100% (347/347), 229.29 KiB | 6.20 MiB/s, done.
Resolving deltas: 100% (229/229), done.


 ***Create Pretrain fiels for multiple sentence docuemnt***

Text corpus description: 
Text corpus contains one document, and the document contains two sentences

Example:
when set dedup_factor as 40, there are total 40 instances created for the single document

In [0]:
!cat "restaurant_review.txt"

I like the mala steamboat a lot.
The chicken rice doesn't taste nice!


In [0]:
!python ./albert/create_pretraining_data.py --input_file "restaurant_review.txt" --output_file "restaurant_review_train" --vocab_file "vocab.txt" --max_seq_length=64 --dupe_factor=40

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

 ***Create Pretrain fiels for single sentence docuemnt***
 
Text corpus description: 
Text corpus contains two document, and each document contains one sentences

Example:
when set dedup_factor as 40, there are total 80 instances created for the text corpus, 40 instances for each document


In [0]:
with open("restaurant_review_single.txt","w") as f:
  f.write("I like the mala steamboat a lot.\n\nThe chicken rice doesn't taste nice!")
!cat "restaurant_review_test.txt"

cat: restaurant_review_test.txt: No such file or directory


In [0]:
!python ./albert/create_pretraining_data.py --input_file "restaurant_review_single.txt" --output_file "restaurant_review_single_train" --vocab_file "vocab.txt" --max_seq_length=64 --dupe_factor=40

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

**Check on data sample details for each batch during Pretrain**

Each instance from the pretrain file, is considered as one data sample in one batch. So if we set dupe_factor = 40, and batch_size = 2, then the same document will appearing in 20 batches


Note: to understand how Albert calculates data sample for each batch considering duplicates when create pretrain files, we copy the exact input building function below and try to replicate the behavior

In [0]:
import tensorflow as tf
name_to_features = {
        "input_ids": tf.FixedLenFeature([64], tf.int64),
        "input_mask": tf.FixedLenFeature([64], tf.int64),
        "segment_ids": tf.FixedLenFeature([64], tf.int64),
        # Note: We keep this feature name `next_sentence_labels` to be
        # compatible with the original data created by lanzhzh@. However, in
        # the ALBERT case it does represent sentence_order_labels.
        "next_sentence_labels": tf.FixedLenFeature([1], tf.int64),
    }
name_to_features.update({
          "masked_lm_positions":
              tf.FixedLenFeature([20], tf.int64),
          "masked_lm_ids":
              tf.FixedLenFeature([20], tf.int64),
          "masked_lm_weights":
              tf.FixedLenFeature([20], tf.float32)})

def _decode_record(record, name_to_features):
  """Decodes a record to a TensorFlow example."""
  example = tf.parse_single_example(record, name_to_features)

  # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
  # So cast all int64 to int32.
  for name in list(example.keys()):
    t = example[name]
    if t.dtype == tf.int64:
      t = tf.to_int32(t)
    example[name] = t

  return example

input_files = tf.gfile.Glob("restaurant_review_train")
d = tf.data.TFRecordDataset(tf.constant(input_files))
d = d.apply(
        tf.data.experimental.map_and_batch_with_legacy_function(
            lambda record: _decode_record(record, name_to_features),
            batch_size=2,
            num_parallel_batches=1,
            drop_remainder=True))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
d

<DatasetV1Adapter shapes: {input_ids: (2, 64), input_mask: (2, 64), masked_lm_ids: (2, 20), masked_lm_positions: (2, 20), masked_lm_weights: (2, 20), next_sentence_labels: (2, 1), segment_ids: (2, 64)}, types: {input_ids: tf.int32, input_mask: tf.int32, masked_lm_ids: tf.int32, masked_lm_positions: tf.int32, masked_lm_weights: tf.float32, next_sentence_labels: tf.int32, segment_ids: tf.int32}>